In [12]:
import csv
from dotenv import load_dotenv
import os
import requests
import json
load_dotenv()

# get ESTA_API_KEY
api_key = os.environ.get('ESTA_API_KEY')

# set url
url = 'https://d09uj3e9kh.execute-api.us-east-1.amazonaws.com/dev/inference'

In [19]:
import csv
import requests

answers = []

with open("data/questions.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    rows = list(reader)

    with open(
        "data/questions_answers.csv", "w", encoding="utf-8", newline="\n"
    ) as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["Question", "Answer"])  # Write header

        for row in rows[0:2]:
            print(row)
            req = requests.get(
                url,
                params={
                    "query": row[0],
                    "sessionId": "123",
                    "collectionName": "levio-hr",
                },
                headers={
                    "x-api-key": api_key,
                    "accept": "application/json",
                    "Content-Type": "application/json",
                },
            )
            response = req.json()  # Assuming the response is JSON
            answer = response[
                "body"
            ]  # Replace 'body' with the key containing the answer in the response
            writer.writerow(
                [row[0], answer]
            )  # Write question and answer to the new CSV

["Qui est le président de l'entreprise Levio?"]
['Quelle est la principale richesse de Levio?']


In [23]:
answers[0].text

" D'après les informations contenues dans le document, le président et fondateur de l'entreprise Levio est François Dion. Plusieurs citations mentionnent que François Dion a fondé Levio en 2013 et qu'il continue d'occuper le rôle de président. Ces citations soulignent également l'importance qu'il accorde à rassembler une équipe motivée autour d'une vision commune de réaliser des projets de transformation numérique d'envergure. On peut donc conclure que le président de l'entreprise Levio est François Dion depuis sa création."